## ATTENTIOM-BIDIRECTIONAL LSTM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

time_series_data = pd.read_csv("PASTE_TRAFFIC_VOLUME_DATA_FILE_PATH_HERE")
time_series_data

In [ ]:
time_series_data.shape

## Data cleaning

In [ ]:

df = time_series_data.drop_duplicates(subset=["TIME"])
df = df.reset_index()
df.shape

In [ ]:
df[100:150]

In [ ]:
df['datetime'] = df['DATE'] + " " + df['TIME']

df.index = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
df

In [ ]:
df[['INTERVAL_COUNT']] = df[['TOTAL']]
df = df[['INTERVAL_COUNT']]
df

In [ ]:
interval_count = df['INTERVAL_COUNT']
plt.plot(interval_count)

## TRAINING LSTM MODEL

In [ ]:
import torch

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
interval_count.isna()

### Data Preprocessing

In [ ]:
def df_to_timeseries(df, window_size):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [[a] for a in df_as_np[i:i+window_size]]
        X.append(row)
        future_val = df_as_np[i+window_size]
        y.append(future_val)
    return X, y

In [ ]:
WINDOW_SIZE = 5

X, y = df_to_timeseries(interval_count, WINDOW_SIZE)
X = np.array(X)
y = np.array(y)
X.shape, y.shape

In [ ]:
from copy import deepcopy as dc

def trasform_to_series(df, n_steps):
    df = dc(df)

    for i in range(1, n_steps+1):
        df[f'INTERVAL_COUNT(t-{i})'] = df['INTERVAL_COUNT'].shift(-i)

    df.dropna(inplace=True)

    return df

timeseries_df = trasform_to_series(df, WINDOW_SIZE)

timeseries_df

In [ ]:
# TRAINING
X_train, y_train = X[:197], y[:197] # 80%
X_val, y_val = X[197:253], y[197:253] # 20%
X_test, y_test = X[253:], y[253:] # 10%

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

In [ ]:
import keras
from keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional




def biderectional_lstm_attention_model(input_shape):

    inputs = keras.Input(shape=input_shape)
    lstm_units = 256
    
    lstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(inputs)
 


    # Attention mechanism
    attention = layers.Dense(1, activation='tanh')(lstm)
    attention = layers.Flatten()(attention)
    attention = layers.Activation('softmax')(attention)
    attention = layers.RepeatVector(lstm_units*2)(attention)
    attention = layers.Permute([2, 1])(attention)
    attention = layers.Multiply()([lstm, attention])
    attention = layers.Lambda(lambda x: tf.reduce_sum(x, axis=1))(attention)

    # Output layer
    outputs = layers.Dense(1)(attention)

    # Compile the model
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError(), 'accuracy'])
    return model

In [ ]:
model = biderectional_lstm_attention_model(input_shape = (WINDOW_SIZE, 1))

In [ ]:
checkpoint = ModelCheckpoint('novel_model/checkpoint.model.keras', save_best_only=True)

In [ ]:
import time

t0 = time.time()

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, callbacks=[checkpoint])

In [ ]:
t1 = time.time()
speed = t1-t0

print(speed)

In [ ]:
# from tensorflow.keras.models import load_model
# novel_model = load_model('novel_model/checkpoint.model.keras', safe_mode=False)

In [ ]:
date_time = df.index[:197]

train_predictions = model.predict(X_train).flatten()


In [ ]:
train_predictions.shape

In [ ]:
train_results = pd.DataFrame(data={'Train Predictions' :train_predictions, 'Actuals' :y_train})
train_results.index = date_time
train_results

In [ ]:
plt.plot(train_results['Train Predictions'], label='Train Prediction')
plt.plot(train_results['Actuals'], label='Actuals')

plt.legend()
plt.title('TRAFFIC VOLUME RESULT COMPARISON')
plt.xlabel('TIME INTERVAL (5 MINUTES)')
plt.ylabel('TRAFFIC VOLUME')
plt.show()

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, root_mean_squared_error

print ('R-Squared:',r2_score(y_train, train_predictions))
print ('Mean Squared Error:', mean_squared_error (y_train, train_predictions))
print ('Mean Absolute Error:', mean_absolute_error (y_train, train_predictions))
print ('Root Mean Squared Error:', root_mean_squared_error (y_train, train_predictions))